# Imports

In [ ]:
import pandas as pd
from hbt_tools import utils_hbt as uh
from hbt_tools import utils as ut
from pathlib import Path

# Functions

In [ ]:
def sum_per_period(periods, ts, date_format, idx_col=0):
    df_res = pd.DataFrame(columns=['von', 'bis', 'summe'])
    for i, p in enumerate(periods):
        date_start = pd.to_datetime(p[0], format=date_format)
        date_start = date_start.tz_localize('Europe/Zurich')
        date_end = pd.to_datetime(p[1], format=date_format)
        date_end = date_end.tz_localize('Europe/Zurich')

        ts_period = ts[(ts.index>=date_start) & (ts.index<=date_end)]
        df_temp = pd.DataFrame({'von': [p[0]], 'bis': [p[1]],
            'summe': [ts_period.iloc[:,idx_col].sum()]})
        
        df_res = pd.concat((df_res, df_temp))
    
    return df_res
#-------------------------------------------------------------------------------

# Parameters

In [ ]:
date_format = '%d.%m.%Y %H:%M'

periods = [
    ("24.09.2022 04:43", "28.09.2022 05:32"),
    ("28.09.2022 05:32", "14.10.2022 16:24"),
    ("14.10.2022 16:24", "21.10.2022 08:36"),
    ("21.10.2022 08:36", "22.10.2022 03:12"),
    ("22.10.2022 03:12", "01.11.2022 07:37"),
    ("01.11.2022 07:37", "16.11.2022 03:18"),
    ("16.11.2022 03:18", "05.02.2023 17:48"),
    ("05.02.2023 17:48", "15.04.2023 12:53"),
    ("15.04.2023 12:53", "01.05.2023 08:25"),
    ("01.05.2023 08:25", "22.06.2023 19:53"),
    ("21.07.2023 08:52", "04.08.2023 14:14"),
    ("04.08.2023 14:14", "23.10.2023 12:53"),
    ("23.10.2023 12:53", "27.10.2023 00:29"),
    ("27.10.2023 00:29", "02.11.2023 13:02"),
]

date_from = periods[0][0]
date_to = periods[-1][-1]

save_dir = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.35-Hagnau\05 Berechnungen Grundlagen\Datenauswertung")

# Regen-Zufluss per Periode

## Regen

In [ ]:
path_dir = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.35-Hagnau\05 Berechnungen Grundlagen\Regendaten")
rain = ut.ts_from_csvs(path_dir)
rain = rain.tz_localize('Europe/Zurich', ambiguous=True)

In [ ]:
df_rain_periods = sum_per_period(periods, rain, date_format)
df_rain_periods

## Zufluss

In [ ]:
id = 1078
q_zu = uh.mdb_getdata(id, date_from, date_to, date_format)

# Compute volume per time-step
q_zu['delta_t'] = q_zu.index
delta_t = q_zu['delta_t'] - q_zu['delta_t'].shift()
print(delta_t.max())
print(delta_t.min())

q_zu['delta_t'] = delta_t
q_zu['volumen'] = q_zu['wert'] * q_zu['delta_t'].dt.total_seconds()

In [ ]:
save_path = save_dir / 'Zulauf_Volumen.csv'
q_zu.to_csv(save_path, sep=';')

In [ ]:
df_zufluss_periods = sum_per_period(periods, q_zu, date_format, idx_col=2)
df_zufluss_periods.iloc[:,2] /= 1000
df_zufluss_periods